In [1]:
from typing import Dict, Text
import numpy as np
import tensorflow as tf
import pandas as pd
import tensorflow_recommenders as tfrs

In [2]:
df = pd.read_csv('Dataset/fiction dataset - Sheet1.csv')
df.head(2)

,fiction_id,title,overview,language,release_date,latest_update,tags,genres,chapter
0,1,Kejahatan Cinta,"Seorang detektif swasta, Anya, sedang menyelid...",Indonesia,2023-10-01,2023-10-06,"detektif,pembunuhan,misteri,cinta,romansa","romansa,misteri",2
1,2,Raja Terakhir,"Di dunia yang dikuasai oleh iblis, seorang pem...",Indonesia,2023-10-02,2023-10-02,"fantasi,aksi,petualangan,pertarungan,kekuatan","fantasi,aksi",1


In [3]:
# read data rating
r_df = pd.read_csv('Dataset/Rating Dataset - Sheet1.csv')
rating_data = r_df[['user_id', 'fiction_id', 'rating']]
rating_data.head(3)

,user_id,fiction_id,rating
0,29,39,0
1,17,44,4
2,94,59,0


Merger

In [4]:
rating_data = rating_data.merge(df[['fiction_id', 'title', 'genres', 'overview']], left_on='fiction_id', right_on='fiction_id')
rating_data.head(3)

,user_id,fiction_id,rating,title,genres,overview
0,29,39,0,Pangeran yang Tertukar,"romansa,petualangan,aksi",Seorang gadis muda menemukan pangeran yang ter...
1,100,39,1,Pangeran yang Tertukar,"romansa,petualangan,aksi",Seorang gadis muda menemukan pangeran yang ter...
2,45,39,2,Pangeran yang Tertukar,"romansa,petualangan,aksi",Seorang gadis muda menemukan pangeran yang ter...


In [5]:
fanfic_df = df[['fiction_id', 'title']]
fanfic_df.head(2)

,fiction_id,title
0,1,Kejahatan Cinta
1,2,Raja Terakhir


In [7]:
rating_data['user_id'] = rating_data['user_id'].astype(str)

ratings = tf.data.Dataset.from_tensor_slices(dict(rating_data[['user_id', 'title', 'rating']]))
fanfics = tf.data.Dataset.from_tensor_slices(dict(fanfic_df[['title']]))

ratings = ratings.map(lambda x: {
    'title': x['title'],
    'user_id': x['user_id'],
    'rating': float(x['rating'])
})

fanfics = fanfics.map(lambda x: x['title'])

In [8]:
print('Total Data: {}'.format(len(ratings)))

Total Data: 5027


In [44]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(5_027, seed=42, reshuffle_each_iteration=False)

train = ratings.take(2_500)
test = ratings.skip(2_500)

In [45]:
fanfic_titles = fanfics.batch(50)
user_ids = ratings.batch(50).map(lambda x: x['user_id'])

In [46]:
unique_fanfics_titles = np.unique(np.concatenate(list(fanfic_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

print('Unique Fanfic: {}'.format(len(unique_fanfics_titles)))
print('Unique users: {}'.format(len(unique_user_ids)))

Unique Fanfic: 99
Unique users: 100


In [59]:
class FanficsModel(tfrs.models.Model):

  def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
    # We take the loss weights in the constructor: this allows us to instantiate
    # several model objects with different loss weights.

    super().__init__()

    embedding_dimension = 64

    # User and fanfic models.
    self.fanfic_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_fanfics_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_fanfics_titles) + 1, embedding_dimension)
    ])
    self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    self.rating_model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(1),
    ])

    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.MeanAbsoluteError()],
    )
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=fanfics.batch(128).map(self.fanfic_model)
        )
    )

    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features['user_id'])
    # And pick out the fanfic features and pass them into the fanfic model.
    fanfic_embeddings = self.fanfic_model(features['title'])
    
    return (
        user_embeddings,
        fanfic_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        # user and fanfic embeddings.
        self.rating_model(
            tf.concat([user_embeddings, fanfic_embeddings], axis=1)
        ),
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    ratings = features.pop('rating')

    user_embeddings, fanfic_embeddings, rating_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(
        labels=ratings,
        predictions=rating_predictions,
    )
    retrieval_loss = self.retrieval_task(user_embeddings, fanfic_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss)

In [60]:
model = FanficsModel(rating_weight=1.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

cached_train = train.shuffle(5_027).batch(50).cache()
cached_test = test.batch(50).cache()

model.fit(cached_train, epochs=10)

Epoch 1/10
50/50 [==============================] - 7s 95ms/step - mean_absolute_error: 1.5719 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0128 - factorized_top_k/top_10_categorical_accuracy: 0.0552 - factorized_top_k/top_50_categorical_accuracy: 0.3540 - factorized_top_k/top_100_categorical_accuracy: 0.9816 - loss: 199.4725 - regularization_loss: 0.0000e+00 - total_loss: 199.4725
Epoch 2/10
50/50 [==============================] - 5s 94ms/step - mean_absolute_error: 1.5222 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0336 - factorized_top_k/top_10_categorical_accuracy: 0.1220 - factorized_top_k/top_50_categorical_accuracy: 0.4664 - factorized_top_k/top_100_categorical_accuracy: 0.9912 - loss: 198.8847 - regularization_loss: 0.0000e+00 - total_loss: 198.8847
Epoch 3/10
50/50 [==============================] - 5s 92ms/step - mean_absolute_error: 1.5101 - factorize

In [64]:
metrics = model.evaluate(cached_test, return_dict=True)

print(f"\nRetrieval top-5 accuracy: {metrics['factorized_top_k/top_5_categorical_accuracy']:.3f}")
print(f"Ranking MAE: {metrics['mean_absolute_error']:.3f}")

51/51 [==============================] - 5s 93ms/step - mean_absolute_error: 1.5539 - factorized_top_k/top_1_categorical_accuracy: 7.9145e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0020 - factorized_top_k/top_10_categorical_accuracy: 0.0024 - factorized_top_k/top_50_categorical_accuracy: 0.4693 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 194.9364 - regularization_loss: 0.0000e+00 - total_loss: 194.9364

Retrieval top-5 accuracy: 0.002
Ranking MAE: 1.554


In [69]:
def predict_fanfic(user, top_n=3):
    # Create a model that takes in raw query features, and
    index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
    # recommends fanfics out of the entire fanfics dataset.
    index.index_from_dataset(
      tf.data.Dataset.zip((fanfics.batch(100), fanfics.batch(100).map(model.fanfic_model)))
    )

    # Get recommendations.
    _, titles = index(tf.constant([str(user)]))
    
    print('Top {} recommendations for user {}:\n'.format(top_n, user))
    for i, title in enumerate(titles[0, :top_n].numpy()):
        print('{}. {}'.format(i+1, title.decode("utf-8")))

def predict_rating(user, fanfic):
    trained_fanfic_embeddings, trained_user_embeddings, predicted_rating = model({
          "userId": np.array([str(user)]),
          "original_title": np.array([fanfic])
      })
    print("Predicted rating for {}: {}".format(fanfic, predicted_rating.numpy()[0][0]))

In [73]:
predict_fanfic(5, 5)

Top 5 recommendations for user 5:

1. Cahaya di Dalam Gelap
2. Petualangan Mencari Mahkota Suci
3. Pangeran yang Tertukar
4. Cinta Tak Terduga
5. Kembali ke Masa Depan
